In [1]:
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import graphviz
import time
import os

# Sklearn
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Tensorflow/Keras
import tensorflow as tf

In [4]:
# Load data
train_df = pd.read_csv("../processed_data/train_df.csv")
train_df['DATE_TIME']= pd.to_datetime(train_df['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

# These were exported to train_df.csv
train_df["HOUR"] = train_df['DATE_TIME'].dt.hour
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv("../processed_data/test_df.csv")
test_df['DATE_TIME']= pd.to_datetime(test_df['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

#These were exported to test_df.csv
test_df["HOUR"] = test_df['DATE_TIME'].dt.hour
test_df.dropna(inplace=True)
test_df.reset_index(drop=True, inplace=True)

mae_results = {}
rmse_results = {}

test_df=test_df.dropna()
train_df=train_df.dropna()


In [115]:
# Day before total yield
df= train_df
max_yield = df.groupby(["SOURCE_KEY","DATE", "TIME"])["AC_POWER"].max().reset_index()
max_yield['AC_POWER'] = max_yield["AC_POWER"]
max_yield

,SOURCE_KEY,DATE,TIME,AC_POWER
0,1BY6WEcLGh8j5v7,2020-05-16,02:00:00,0.0
1,1BY6WEcLGh8j5v7,2020-05-16,02:15:00,0.0
2,1BY6WEcLGh8j5v7,2020-05-16,02:30:00,0.0
3,1BY6WEcLGh8j5v7,2020-05-16,02:45:00,0.0
4,1BY6WEcLGh8j5v7,2020-05-16,03:00:00,0.0
...,...,...,...,...
105127,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0
105128,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0
105129,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0
105130,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0


In [116]:
from datetime import datetime
max_yield['DATE']= pd.to_datetime(max_yield['DATE'])
max_yield['YDAY_DATE']=max_yield['DATE']+timedelta(1)
max_yield

,SOURCE_KEY,DATE,TIME,AC_POWER,YDAY_DATE
0,1BY6WEcLGh8j5v7,2020-05-16,02:00:00,0.0,2020-05-17
1,1BY6WEcLGh8j5v7,2020-05-16,02:15:00,0.0,2020-05-17
2,1BY6WEcLGh8j5v7,2020-05-16,02:30:00,0.0,2020-05-17
3,1BY6WEcLGh8j5v7,2020-05-16,02:45:00,0.0,2020-05-17
4,1BY6WEcLGh8j5v7,2020-05-16,03:00:00,0.0,2020-05-17
...,...,...,...,...,...
105127,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0,2020-06-12
105128,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0,2020-06-12
105129,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0,2020-06-12
105130,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0,2020-06-12


In [117]:
import copy
yield2=copy.deepcopy(max_yield)
yield2['DATE']=yield2['YDAY_DATE']
del yield2['YDAY_DATE']
yield2

,SOURCE_KEY,DATE,TIME,AC_POWER
0,1BY6WEcLGh8j5v7,2020-05-17,02:00:00,0.0
1,1BY6WEcLGh8j5v7,2020-05-17,02:15:00,0.0
2,1BY6WEcLGh8j5v7,2020-05-17,02:30:00,0.0
3,1BY6WEcLGh8j5v7,2020-05-17,02:45:00,0.0
4,1BY6WEcLGh8j5v7,2020-05-17,03:00:00,0.0
...,...,...,...,...
105127,zVJPv84UY57bAof,2020-06-12,22:45:00,0.0
105128,zVJPv84UY57bAof,2020-06-12,23:00:00,0.0
105129,zVJPv84UY57bAof,2020-06-12,23:15:00,0.0
105130,zVJPv84UY57bAof,2020-06-12,23:30:00,0.0


In [119]:
# yield2.join(max_yield.set_index(['SOURCE_KEY','HOUR']), on='DATE')
yield3 = pd.merge(max_yield, yield2, on=['SOURCE_KEY', 'TIME', "DATE"], suffixes=('_dayof', '_daybefore'))
yield3

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-05-17,02:00:00,0.0,2020-05-18,0.0
1,1BY6WEcLGh8j5v7,2020-05-17,02:15:00,0.0,2020-05-18,0.0
2,1BY6WEcLGh8j5v7,2020-05-17,02:30:00,0.0,2020-05-18,0.0
3,1BY6WEcLGh8j5v7,2020-05-17,02:45:00,0.0,2020-05-18,0.0
4,1BY6WEcLGh8j5v7,2020-05-17,03:00:00,0.0,2020-05-18,0.0
...,...,...,...,...,...,...
97450,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0,2020-06-12,0.0
97451,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0,2020-06-12,0.0
97452,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0,2020-06-12,0.0
97453,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0,2020-06-12,0.0


In [67]:
yield3[20:60]

,SOURCE_KEY,DATE,HOUR,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
20,1BY6WEcLGh8j5v7,2020-05-17,22,0.000000,2020-05-18,0.000000
21,1BY6WEcLGh8j5v7,2020-05-17,23,0.000000,2020-05-18,0.000000
22,1BY6WEcLGh8j5v7,2020-05-18,0,0.000000,2020-05-19,0.000000
23,1BY6WEcLGh8j5v7,2020-05-18,1,0.000000,2020-05-19,0.000000
24,1BY6WEcLGh8j5v7,2020-05-18,2,0.000000,2020-05-19,0.000000
25,1BY6WEcLGh8j5v7,2020-05-18,3,0.000000,2020-05-19,0.000000
26,1BY6WEcLGh8j5v7,2020-05-18,4,0.000000,2020-05-19,0.000000
27,1BY6WEcLGh8j5v7,2020-05-18,5,0.000000,2020-05-19,0.000000
28,1BY6WEcLGh8j5v7,2020-05-18,6,104.671429,2020-05-19,78.262500
29,1BY6WEcLGh8j5v7,2020-05-18,7,304.362500,2020-05-19,401.487500


In [87]:
max_yield[22000:22040]

,SOURCE_KEY,DATE,HOUR,AC_POWER,YDAY_DATE
22000,uHbuxQJl8lW7ozc,2020-05-26,9,984.200000,2020-05-27
22001,uHbuxQJl8lW7ozc,2020-05-26,10,1100.414286,2020-05-27
22002,uHbuxQJl8lW7ozc,2020-05-26,11,1093.871429,2020-05-27
22003,uHbuxQJl8lW7ozc,2020-05-26,12,1293.187500,2020-05-27
22004,uHbuxQJl8lW7ozc,2020-05-26,13,1241.042857,2020-05-27
22005,uHbuxQJl8lW7ozc,2020-05-26,14,867.437500,2020-05-27
22006,uHbuxQJl8lW7ozc,2020-05-26,15,691.071429,2020-05-27
22007,uHbuxQJl8lW7ozc,2020-05-26,16,629.642857,2020-05-27
22008,uHbuxQJl8lW7ozc,2020-05-26,17,446.942857,2020-05-27
22009,uHbuxQJl8lW7ozc,2020-05-26,18,35.700000,2020-05-27


In [86]:
yield3[yield3.SOURCE_KEY=='uHbuxQJl8lW7ozc'][200:240]

,SOURCE_KEY,DATE,HOUR,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
20592,uHbuxQJl8lW7ozc,2020-05-26,10,1100.414286,2020-05-27,1142.462500
20593,uHbuxQJl8lW7ozc,2020-05-26,11,1093.871429,2020-05-27,1220.271429
20594,uHbuxQJl8lW7ozc,2020-05-26,12,1293.187500,2020-05-27,1236.485714
20595,uHbuxQJl8lW7ozc,2020-05-26,13,1241.042857,2020-05-27,1219.400000
20596,uHbuxQJl8lW7ozc,2020-05-26,14,867.437500,2020-05-27,1137.725000
20597,uHbuxQJl8lW7ozc,2020-05-26,15,691.071429,2020-05-27,1016.900000
20598,uHbuxQJl8lW7ozc,2020-05-26,16,629.642857,2020-05-27,731.728571
20599,uHbuxQJl8lW7ozc,2020-05-26,17,446.942857,2020-05-27,111.642857
20600,uHbuxQJl8lW7ozc,2020-05-26,18,35.700000,2020-05-27,42.128571
20601,uHbuxQJl8lW7ozc,2020-05-26,19,0.000000,2020-05-27,0.000000


In [120]:
del yield3['YDAY_DATE']

In [121]:
yield3

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-05-17,02:00:00,0.0,0.0
1,1BY6WEcLGh8j5v7,2020-05-17,02:15:00,0.0,0.0
2,1BY6WEcLGh8j5v7,2020-05-17,02:30:00,0.0,0.0
3,1BY6WEcLGh8j5v7,2020-05-17,02:45:00,0.0,0.0
4,1BY6WEcLGh8j5v7,2020-05-17,03:00:00,0.0,0.0
...,...,...,...,...,...
97450,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0,0.0
97451,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0,0.0
97452,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0,0.0
97453,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0,0.0


## do for test_df

In [122]:
# Day before total yield
df= test_df
test_yield = df.groupby(["SOURCE_KEY","DATE", "TIME"])["AC_POWER"].max().reset_index()
test_yield['AC_POWER'] = test_yield["AC_POWER"]
test_yield

,SOURCE_KEY,DATE,TIME,AC_POWER
0,1BY6WEcLGh8j5v7,2020-06-13,00:00:00,0.0
1,1BY6WEcLGh8j5v7,2020-06-13,00:15:00,0.0
2,1BY6WEcLGh8j5v7,2020-06-13,00:30:00,0.0
3,1BY6WEcLGh8j5v7,2020-06-13,00:45:00,0.0
4,1BY6WEcLGh8j5v7,2020-06-13,01:00:00,0.0
...,...,...,...,...
23183,zVJPv84UY57bAof,2020-06-17,22:45:00,0.0
23184,zVJPv84UY57bAof,2020-06-17,23:00:00,0.0
23185,zVJPv84UY57bAof,2020-06-17,23:15:00,0.0
23186,zVJPv84UY57bAof,2020-06-17,23:30:00,0.0


In [123]:
test_yield['DATE']= pd.to_datetime(test_yield['DATE'])
test_yield['YDAY_DATE']=test_yield['DATE']+timedelta(1)
test_yield

,SOURCE_KEY,DATE,TIME,AC_POWER,YDAY_DATE
0,1BY6WEcLGh8j5v7,2020-06-13,00:00:00,0.0,2020-06-14
1,1BY6WEcLGh8j5v7,2020-06-13,00:15:00,0.0,2020-06-14
2,1BY6WEcLGh8j5v7,2020-06-13,00:30:00,0.0,2020-06-14
3,1BY6WEcLGh8j5v7,2020-06-13,00:45:00,0.0,2020-06-14
4,1BY6WEcLGh8j5v7,2020-06-13,01:00:00,0.0,2020-06-14
...,...,...,...,...,...
23183,zVJPv84UY57bAof,2020-06-17,22:45:00,0.0,2020-06-18
23184,zVJPv84UY57bAof,2020-06-17,23:00:00,0.0,2020-06-18
23185,zVJPv84UY57bAof,2020-06-17,23:15:00,0.0,2020-06-18
23186,zVJPv84UY57bAof,2020-06-17,23:30:00,0.0,2020-06-18


In [124]:
test_yield_2=copy.deepcopy(test_yield)
test_yield_2['DATE']=test_yield_2['YDAY_DATE']
del test_yield_2['YDAY_DATE']
test_yield_2

,SOURCE_KEY,DATE,TIME,AC_POWER
0,1BY6WEcLGh8j5v7,2020-06-14,00:00:00,0.0
1,1BY6WEcLGh8j5v7,2020-06-14,00:15:00,0.0
2,1BY6WEcLGh8j5v7,2020-06-14,00:30:00,0.0
3,1BY6WEcLGh8j5v7,2020-06-14,00:45:00,0.0
4,1BY6WEcLGh8j5v7,2020-06-14,01:00:00,0.0
...,...,...,...,...
23183,zVJPv84UY57bAof,2020-06-18,22:45:00,0.0
23184,zVJPv84UY57bAof,2020-06-18,23:00:00,0.0
23185,zVJPv84UY57bAof,2020-06-18,23:15:00,0.0
23186,zVJPv84UY57bAof,2020-06-18,23:30:00,0.0


In [125]:
# yield2.join(max_yield.set_index(['SOURCE_KEY','HOUR']), on='DATE')
test_yield_3 = pd.merge(test_yield, test_yield_2, on=['SOURCE_KEY', 'TIME', "DATE"], suffixes=('_dayof', '_daybefore'))
test_yield_3

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-06-14,00:00:00,0.0,2020-06-15,0.0
1,1BY6WEcLGh8j5v7,2020-06-14,00:15:00,0.0,2020-06-15,0.0
2,1BY6WEcLGh8j5v7,2020-06-14,00:30:00,0.0,2020-06-15,0.0
3,1BY6WEcLGh8j5v7,2020-06-14,00:45:00,0.0,2020-06-15,0.0
4,1BY6WEcLGh8j5v7,2020-06-14,01:00:00,0.0,2020-06-15,0.0
...,...,...,...,...,...,...
18959,zVJPv84UY57bAof,2020-06-17,22:45:00,0.0,2020-06-18,0.0
18960,zVJPv84UY57bAof,2020-06-17,23:00:00,0.0,2020-06-18,0.0
18961,zVJPv84UY57bAof,2020-06-17,23:15:00,0.0,2020-06-18,0.0
18962,zVJPv84UY57bAof,2020-06-17,23:30:00,0.0,2020-06-18,0.0


# Write to file

In [126]:
test_yield_3

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-06-14,00:00:00,0.0,2020-06-15,0.0
1,1BY6WEcLGh8j5v7,2020-06-14,00:15:00,0.0,2020-06-15,0.0
2,1BY6WEcLGh8j5v7,2020-06-14,00:30:00,0.0,2020-06-15,0.0
3,1BY6WEcLGh8j5v7,2020-06-14,00:45:00,0.0,2020-06-15,0.0
4,1BY6WEcLGh8j5v7,2020-06-14,01:00:00,0.0,2020-06-15,0.0
...,...,...,...,...,...,...
18959,zVJPv84UY57bAof,2020-06-17,22:45:00,0.0,2020-06-18,0.0
18960,zVJPv84UY57bAof,2020-06-17,23:00:00,0.0,2020-06-18,0.0
18961,zVJPv84UY57bAof,2020-06-17,23:15:00,0.0,2020-06-18,0.0
18962,zVJPv84UY57bAof,2020-06-17,23:30:00,0.0,2020-06-18,0.0


In [127]:
yield3

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-05-17,02:00:00,0.0,0.0
1,1BY6WEcLGh8j5v7,2020-05-17,02:15:00,0.0,0.0
2,1BY6WEcLGh8j5v7,2020-05-17,02:30:00,0.0,0.0
3,1BY6WEcLGh8j5v7,2020-05-17,02:45:00,0.0,0.0
4,1BY6WEcLGh8j5v7,2020-05-17,03:00:00,0.0,0.0
...,...,...,...,...,...
97450,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0,0.0
97451,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0,0.0
97452,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0,0.0
97453,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0,0.0


In [136]:
train_AC_lagged = copy.deepcopy(yield3)
test_AC_lagged = copy.deepcopy(test_yield_3)
train_AC_lagged

,SOURCE_KEY,DATE,TIME,AC_POWER_dayof,AC_POWER_daybefore
0,1BY6WEcLGh8j5v7,2020-05-17,02:00:00,0.0,0.0
1,1BY6WEcLGh8j5v7,2020-05-17,02:15:00,0.0,0.0
2,1BY6WEcLGh8j5v7,2020-05-17,02:30:00,0.0,0.0
3,1BY6WEcLGh8j5v7,2020-05-17,02:45:00,0.0,0.0
4,1BY6WEcLGh8j5v7,2020-05-17,03:00:00,0.0,0.0
...,...,...,...,...,...
97450,zVJPv84UY57bAof,2020-06-11,22:45:00,0.0,0.0
97451,zVJPv84UY57bAof,2020-06-11,23:00:00,0.0,0.0
97452,zVJPv84UY57bAof,2020-06-11,23:15:00,0.0,0.0
97453,zVJPv84UY57bAof,2020-06-11,23:30:00,0.0,0.0


In [129]:
train_df

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,WEATHER_SOURCE,DATE,TIME,is_daytime,yesterday_max_daily_yield,HOUR
0,2020-05-16 01:45:00,4136001,LlT2YUhhzqhg5Sw,0.000000,0.000000,0.000000,2.826020e+08,25.954583,23.974197,0.000000,iq8k7ZNt4Mwm3w0,2020-05-16,01:45:00,0,9234.0,1
1,2020-05-16 15:30:00,4136001,NgDl19wMapZy17u,582.106667,570.446667,7294.266667,1.115294e+08,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,9475.0,15
2,2020-05-16 15:30:00,4136001,Mx2yZCDsyf6DPfv,597.557143,585.471429,5334.571429,2.466781e+06,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,7800.0,15
3,2020-05-16 15:30:00,4136001,LlT2YUhhzqhg5Sw,547.846667,537.066667,7085.133333,2.826091e+08,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,9234.0,15
4,2020-05-16 15:30:00,4136001,LYwnQax7tkwH5Cb,559.340000,548.213333,2717.400000,1.794970e+09,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,8458.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105127,2020-06-11 16:45:00,4136001,q49J1IKaHRwDQnt,109.220000,106.166667,3681.133333,4.866221e+05,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,5303.0,16
105128,2020-06-11 16:45:00,4136001,oZZkBaNadn6DNKz,106.935714,103.935714,3785.928571,1.708260e+09,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,5460.0,16
105129,2020-06-11 16:45:00,4136001,oZ35aAeoifZaQzV,112.306667,109.166667,3824.333333,1.660165e+09,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,8094.0,16
105130,2020-06-11 16:45:00,4136001,mqwcsP2rE7J0TFp,113.253333,110.113333,3742.466667,5.937821e+08,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,2636.0,16


In [131]:
train_df['DATE']= pd.to_datetime(train_df['DATE'])
train_df_new = pd.merge(train_df, train_AC_lagged, on=['SOURCE_KEY', 'TIME', "DATE"], suffixes=('og', 'new'), how='left')
train_df_new

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,WEATHER_SOURCE,DATE,TIME,is_daytime,yesterday_max_daily_yield,HOUR,AC_POWER_dayof,AC_POWER_daybefore
0,2020-05-16 01:45:00,4136001,LlT2YUhhzqhg5Sw,0.000000,0.000000,0.000000,2.826020e+08,25.954583,23.974197,0.000000,iq8k7ZNt4Mwm3w0,2020-05-16,01:45:00,0,9234.0,1,NaN,NaN
1,2020-05-16 15:30:00,4136001,NgDl19wMapZy17u,582.106667,570.446667,7294.266667,1.115294e+08,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,9475.0,15,NaN,NaN
2,2020-05-16 15:30:00,4136001,Mx2yZCDsyf6DPfv,597.557143,585.471429,5334.571429,2.466781e+06,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,7800.0,15,NaN,NaN
3,2020-05-16 15:30:00,4136001,LlT2YUhhzqhg5Sw,547.846667,537.066667,7085.133333,2.826091e+08,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,9234.0,15,NaN,NaN
4,2020-05-16 15:30:00,4136001,LYwnQax7tkwH5Cb,559.340000,548.213333,2717.400000,1.794970e+09,34.537255,42.353539,0.393412,iq8k7ZNt4Mwm3w0,2020-05-16,15:30:00,1,8458.0,15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105127,2020-06-11 16:45:00,4136001,q49J1IKaHRwDQnt,109.220000,106.166667,3681.133333,4.866221e+05,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,5303.0,16,106.166667,225.240000
105128,2020-06-11 16:45:00,4136001,oZZkBaNadn6DNKz,106.935714,103.935714,3785.928571,1.708260e+09,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,5460.0,16,103.935714,209.386667
105129,2020-06-11 16:45:00,4136001,oZ35aAeoifZaQzV,112.306667,109.166667,3824.333333,1.660165e+09,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,8094.0,16,109.166667,215.126667
105130,2020-06-11 16:45:00,4136001,mqwcsP2rE7J0TFp,113.253333,110.113333,3742.466667,5.937821e+08,25.617999,25.562179,0.068218,iq8k7ZNt4Mwm3w0,2020-06-11,16:45:00,1,2636.0,16,110.113333,228.228571


In [133]:
train_df_new[train_df_new.SOURCE_KEY=='LlT2YUhhzqhg5Sw'].sort_values(by=['DATE_TIME'])[120:130]

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,WEATHER_SOURCE,DATE,TIME,is_daytime,yesterday_max_daily_yield,HOUR,AC_POWER_dayof,AC_POWER_daybefore
7273,2020-05-17 06:00:00,4136001,LlT2YUhhzqhg5Sw,3.406667,3.286667,0.066667,2.826096e+08,24.319195,23.056785,0.005540,iq8k7ZNt4Mwm3w0,2020-05-17,06:00:00,1,7542.0,6,3.286667,13.792857
7251,2020-05-17 06:15:00,4136001,LlT2YUhhzqhg5Sw,24.133333,23.306667,3.533333,2.826096e+08,25.056465,24.004123,0.018370,iq8k7ZNt4Mwm3w0,2020-05-17,06:15:00,1,7542.0,6,23.306667,44.400000
7427,2020-05-17 06:30:00,4136001,LlT2YUhhzqhg5Sw,36.146667,34.920000,11.200000,2.826096e+08,25.291162,24.822537,0.026292,iq8k7ZNt4Mwm3w0,2020-05-17,06:30:00,1,7542.0,6,34.920000,82.306667
7471,2020-05-17 06:45:00,4136001,LlT2YUhhzqhg5Sw,42.946667,41.526667,20.466667,2.826096e+08,25.523689,25.355253,0.031181,iq8k7ZNt4Mwm3w0,2020-05-17,06:45:00,1,7542.0,6,41.526667,122.113333
7449,2020-05-17 07:00:00,4136001,LlT2YUhhzqhg5Sw,68.085714,65.957143,33.500000,2.826096e+08,25.723396,25.946431,0.047667,iq8k7ZNt4Mwm3w0,2020-05-17,07:00:00,1,7542.0,7,65.957143,161.793333
7361,2020-05-17 07:15:00,4136001,LlT2YUhhzqhg5Sw,99.620000,96.766667,52.800000,2.826096e+08,26.135726,27.001655,0.069437,iq8k7ZNt4Mwm3w0,2020-05-17,07:15:00,1,7542.0,7,96.766667,244.842857
7405,2020-05-17 07:30:00,4136001,LlT2YUhhzqhg5Sw,180.186667,176.386667,85.466667,2.826097e+08,26.674718,28.918446,0.127058,iq8k7ZNt4Mwm3w0,2020-05-17,07:30:00,1,7542.0,7,176.386667,373.320000
7383,2020-05-17 07:45:00,4136001,LlT2YUhhzqhg5Sw,321.906667,316.093333,146.333333,2.826097e+08,27.605172,32.495211,0.235718,iq8k7ZNt4Mwm3w0,2020-05-17,07:45:00,1,7542.0,7,316.093333,376.066667
8087,2020-05-17 08:00:00,4136001,LlT2YUhhzqhg5Sw,566.080000,554.846667,250.533333,2.826098e+08,28.544006,37.891594,0.430667,iq8k7ZNt4Mwm3w0,2020-05-17,08:00:00,1,7542.0,8,554.846667,539.240000
8131,2020-05-17 08:15:00,4136001,LlT2YUhhzqhg5Sw,607.685714,595.371429,404.000000,2.826100e+08,29.212375,43.304464,0.461763,iq8k7ZNt4Mwm3w0,2020-05-17,08:15:00,1,7542.0,8,595.371429,606.200000


In [135]:
train_df_new[train_df_new.SOURCE_KEY=='LlT2YUhhzqhg5Sw'].sort_values(by=['DATE_TIME'])[24:30]

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,WEATHER_SOURCE,DATE,TIME,is_daytime,yesterday_max_daily_yield,HOUR,AC_POWER_dayof,AC_POWER_daybefore
2003,2020-05-16 06:00:00,4136001,LlT2YUhhzqhg5Sw,14.300000,13.792857,0.857143,2.826020e+08,24.044605,22.218803,0.013039,iq8k7ZNt4Mwm3w0,2020-05-16,06:00:00,1,9234.0,6,NaN,NaN
2013,2020-05-16 06:15:00,4136001,LlT2YUhhzqhg5Sw,45.913333,44.400000,7.933333,2.826021e+08,23.967376,22.682919,0.035180,iq8k7ZNt4Mwm3w0,2020-05-16,06:15:00,1,9234.0,6,NaN,NaN
2206,2020-05-16 06:30:00,4136001,LlT2YUhhzqhg5Sw,84.806667,82.306667,23.600000,2.826021e+08,24.587146,23.915495,0.060926,iq8k7ZNt4Mwm3w0,2020-05-16,06:30:00,1,9234.0,6,NaN,NaN
2203,2020-05-16 06:45:00,4136001,LlT2YUhhzqhg5Sw,125.520000,122.113333,49.400000,2.826021e+08,25.354186,25.459056,0.087557,iq8k7ZNt4Mwm3w0,2020-05-16,06:45:00,1,9234.0,6,NaN,NaN
2215,2020-05-16 07:00:00,4136001,LlT2YUhhzqhg5Sw,165.120000,161.793333,85.800000,2.826021e+08,25.971491,27.421957,0.127116,iq8k7ZNt4Mwm3w0,2020-05-16,07:00:00,1,9234.0,7,NaN,NaN
2131,2020-05-16 07:15:00,4136001,LlT2YUhhzqhg5Sw,249.514286,244.842857,131.928571,2.826022e+08,26.566503,28.132564,0.197787,iq8k7ZNt4Mwm3w0,2020-05-16,07:15:00,1,9234.0,7,NaN,NaN


In [139]:
test_df['DATE']= pd.to_datetime(test_df['DATE'])
test_df_new = pd.merge(test_df, test_AC_lagged, on=['SOURCE_KEY', 'TIME', "DATE"], suffixes=('og', 'new'), how='left')
test_df_new

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,WEATHER_SOURCE,DATE,TIME,is_daytime,yesterday_max_daily_yield,HOUR,AC_POWER_dayof,YDAY_DATE,AC_POWER_daybefore
0,2020-06-12 02:30:00,4136001,LYwnQax7tkwH5Cb,0.000000,0.000000,3718.000000,1.795083e+09,22.610987,22.146187,0.000000,iq8k7ZNt4Mwm3w0,2020-06-12,02:30:00,0,3718.0,2,NaN,NaT,NaN
1,2020-06-12 23:45:00,4136001,xoJJ8DcxJEcupym,0.000000,0.000000,4788.000000,2.093055e+08,23.099230,22.757666,0.000000,iq8k7ZNt4Mwm3w0,2020-06-12,23:45:00,0,3971.0,23,NaN,NaT,NaN
2,2020-06-12 07:45:00,4136001,WcxssY2VbP4hApt,267.833333,262.780000,194.800000,1.818769e+08,24.554531,27.705916,0.175870,iq8k7ZNt4Mwm3w0,2020-06-12,07:45:00,1,3947.0,7,NaN,NaT,NaN
3,2020-06-12 07:45:00,4136001,mqwcsP2rE7J0TFp,284.813333,279.646667,201.533333,5.937824e+08,24.554531,27.705916,0.175870,iq8k7ZNt4Mwm3w0,2020-06-12,07:45:00,1,3877.0,7,NaN,NaT,NaN
4,2020-06-12 07:45:00,4136001,oZ35aAeoifZaQzV,262.600000,257.593333,195.866667,1.660165e+09,24.554531,27.705916,0.175870,iq8k7ZNt4Mwm3w0,2020-06-12,07:45:00,1,5585.0,7,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23183,2020-06-17 16:15:00,4135001,bvBOhCH3iADSZry,341.128571,334.957143,5215.000000,6.538764e+06,24.112787,28.996932,0.235078,HmiyD2TTLFNqkNe,2020-06-17,16:15:00,1,5757.0,16,334.957143,2020-06-18,317.228571
23184,2020-06-17 16:15:00,4135001,iCRJl6heRkivqQ3,351.928571,345.714286,5693.571429,7.426000e+06,24.112787,28.996932,0.235078,HmiyD2TTLFNqkNe,2020-06-17,16:15:00,1,6305.0,16,345.714286,2020-06-18,354.285714
23185,2020-06-17 16:15:00,4135001,ih0vzX44oOqAx2f,350.714286,344.485714,5497.857143,6.425869e+06,24.112787,28.996932,0.235078,HmiyD2TTLFNqkNe,2020-06-17,16:15:00,1,6124.0,16,344.485714,2020-06-18,337.128571
23186,2020-06-17 16:15:00,4135001,rGa61gmuvPhdLxV,360.528571,353.985714,5688.428571,7.356635e+06,24.112787,28.996932,0.235078,HmiyD2TTLFNqkNe,2020-06-17,16:15:00,1,6243.0,16,353.985714,2020-06-18,358.457143


In [142]:
test_df_new.to_csv('test_df_aclagged.csv')  
train_df_new.to_csv('train_df_aclagged.csv')  